# Test Dataset 2: 10 minutes of data from every station in the PNSN, c. October 2023
This dataset includes every PNSN recording that conforms to channel code [BHE][HN][ZNE12] for the 3 minutes preceeding and 7 minutes following the
M4.3 earthquake near Port Townsend, WA on October 8th 2023 (2023-10-09T02:21:08 UTC)
Use the `Client` class methods to conduct this query

:auth: Nathan T. Stevens  
:email: ntsteven (at) uw.edu  
:org: Pacific Northwest Seismic Network  
:license: MIT (2023)  


:references:  
Ni, Yiyu, Alexander Hutko, Francesca Skene, Marine Denolle, Stephen Malone, Paul Bodin, Renate Hartog, and Amy Wright. “Curated Pacific Northwest AI-Ready Seismic Dataset.” Seismica 2, no. 1 (May 8, 2023). https://doi.org/10.26443/seismica.v2i1.368.




In [1]:
import os
import sys
import pandas as pd
from obspy import UTCDateTime, Stream
from obspy.clients.fdsn import Client
from obspy.geodetics import kilometers2degrees as kms2degs
from tqdm import tqdm
import plotly.express as px


# from time import time
# # Import repo-specific data holders
# sys.path.append('..')
# from classes.EventMiniDB import EventMiniDB, MiniCat

In [2]:
# Define data save root directory (in this case, we go up a level and right back into 'data')
OUT_ROOT = os.path.join('..','data')
# Define root location for local PNW_Store data subset (see Ni et al., 2023)
# PNW_ROOT = os.path.join('/Volumes','TheWall','PNW_Store_Local')

In [3]:
# Start creating subdirectory structure if it doesn't already exist
set1_root = os.path.join(OUT_ROOT,'test_dataset_2')
try: 
    os.mkdir(set1_root)
except FileExistsError:
    pass

# Time Bounds
TS = UTCDateTime(2023,10,9,2,18)
TE = TS + 600
# Compose channel list
chan_list = []
# Get band codes 'B', 'H', and 'E'
for _bc in 'BHE':
    # Get instrument codes 'H' and 'N'
    for _ic in 'HN':
        # Get all reasonable component codes
        for _cc in 'ZNE123':
            # Add unique combinations to list
            if _bc + _ic + _cc not in chan_list:
                chan_list.append(_bc + _ic + _cc)

# Mash into a single long string to pass to client.get_stations()
cstr = chan_list[0]
for _c in chan_list[1:]:
    cstr += f',{_c}'

inv_kwargs = {'network':'UW',
              'starttime': TS, 'endtime': TE,
              'channel': cstr, 'level': 'response'}

# Initialize client
client = Client("IRIS")
# Run Station Query
print('Loading inventory from Client')
inv = client.get_stations(**inv_kwargs)
print('Saving inventory to disk')
# Write station inventory to disk
inv.write(os.path.join(set1_root, 'stations.xml'),format='STATIONXML')

# Convert obspy.Inventory into pandas.DataFrame
holder = []
for _n in inv.networks:
    for _s in _n.stations:
        #       net      sta      nchan              lat          lon           Elev_m
        line = [_n.code, _s.code, len(_s.channels), _s.latitude, _s.longitude, _s.elevation]
        if _s.start_date >= TS:
            start_timestamp = pd.Timestamp(_s.start_date.isoformat())
        else:
            start_timestamp = pd.Timestamp(TS.isoformat())
            
        if _s.end_date is None:
            end_timestamp = pd.Timestamp(TE.isoformat())
        elif _s.end_date < TE:
            end_timestamp = pd.Timestamp(_s.end_date.isoformat())
        else:
            end_timestamp = pd.Timestamp(TE.isoformat())
            
        frac_complete = (end_timestamp - start_timestamp).total_seconds()/(TE - TS)
        line += [start_timestamp, end_timestamp, frac_complete]
        holder.append(line)

df_inv = pd.DataFrame(holder, columns=['net','sta','nchan','lat','lon','elev_m','ts_start','ts_end','coverage_frac'])
display(df_inv)
df_inv.to_csv(os.path.join(set1_root, 'stations_summary.csv'), header=True, index=False)


Loading inventory from Client
Saving inventory to disk


,net,sta,nchan,lat,lon,elev_m,ts_start,ts_end,coverage_frac
0,UW,ABER,6,46.34275,-123.099230,786.3,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0
1,UW,AGNW,6,46.33332,-118.679420,355.0,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0
2,UW,ALCT,3,47.64690,-122.037700,55.0,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0
3,UW,ALKI,3,47.57510,-122.417600,1.0,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0
4,UW,ALLI,3,47.32005,-122.915720,77.5,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0
...,...,...,...,...,...,...,...,...,...
528,UW,YA2,1,46.52652,-120.531181,652.0,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0
529,UW,YACH,3,44.31500,-124.096600,217.0,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0
530,UW,YACT,6,45.93250,-122.419300,214.0,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0
531,UW,YELM,6,46.94016,-122.590810,102.9,2023-10-09 02:18:00,2023-10-09 02:28:00,1.0


In [4]:
# Do some visualization with plotly
fig = px.scatter_mapbox(df_inv,lat='lat',lon='lon',color='nchan',size='nchan',
                        hover_name='sta',hover_data=['net','sta','nchan','coverage_frac'],
                        height=800,width=800, zoom=9, text='sta')#, opacity='coverage_frac')
fig.update_layout(mapbox_style='carto-positron')
fig.update_geos(resolution=110)

In [5]:
print('Starting to pull waveform data')
NSLBI_list = []
wfdisc_holder = []
for _n in inv.networks:
    print(f'Running stattion data pulls for network {_n.code}')
    for _s in tqdm(_n.stations):
        for _c in _s.channels:
            _nslbi = f'{_n.code}.{_s.code}.{_c.location_code}.{_c.code[:2]}?'
            if _nslbi not in NSLBI_list:
                NSLBI_list.append(_nslbi)
                _ts = TS
                # Define save subdirectory structure ROOT/{NetCode}/{StaCode}
                _save_dir = os.path.join(set1_root, _n.code, _s.code)

                # Form kwargs
                wfq_kwargs = dict(zip(['network','station','location','channel'],_nslbi.split('.')))
                # Run data pull
                try:
                    st = client.get_waveforms(**wfq_kwargs, starttime=_ts, endtime=_ts + 3600)
                except:
                    st = Stream()

                if len(st) > 0:
                    wf_ts = min([_tr.stats.starttime.isoformat() for _tr in st])
                    wf_te = max([_tr.stats.endtime.isoformat() for _tr in st])
                    try:
                        os.makedirs(_save_dir)
                    except FileExistsError:
                        pass
                    # _save_dir/{NetCode}.{StaCode}.{LocCode}.{BandChar}{InstChar}.{Year}.{Jdate:03d}.mseed
                    _save_file = f'{_n.code:s}.{_s.code:s}.{_c.location_code:s}.{_c.code[:2]:s}?.{_ts.year:d}.{_ts.julday:03d}.mseed'
                    _save_pf = os.path.join(_save_dir,_save_file)
                    if ~os.path.exists(_save_pf):
                        st.write(_save_pf, fmt='MSEED')
                    wfdisc_line = [_s.code,_c.code[:2],wf_ts, wf_te, _c.sample_rate, _save_dir, _save_file, pd.Timestamp.now()]
                    wfdisc_holder.append(wfdisc_line)

df_wfdisc = pd.DataFrame(wfdisc_holder,columns=['sta','bandinst','time','endtime','samprate','dir','dfile','lddate'])
df_wfdisc.to_csv(os.path.join(set1_root,'wfdisc.csv'),header=True,index=True)

Starting to pull waveform data
Running stattion data pulls for network UW


  0%|          | 0/533 [00:00<?, ?it/s]

100%|██████████| 533/533 [16:22<00:00,  1.84s/it]
